# Setup

In [ ]:
import os
import time
import json
import requests
from google import genai
from IPython.display import display, Markdown

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

# Setup for REST API calls
API_KEY = os.getenv('GEMINI_API_KEY')
MODEL = "gemini-3-flash-preview"
BASE_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL}"

# URLs for different response modes
url_standard = f"{BASE_URL}:generateContent?key={API_KEY}"
url_stream = f"{BASE_URL}:streamGenerateContent?key={API_KEY}"

In [2]:
def response(payload, stream=False):
    """Send a request to Gemini API and get the response.
    
    Args:
        payload: The request payload
        stream: If True, returns streaming response; if False, returns standard response
    
    Returns:
        For standard: Full response JSON
        For streaming: Generator yielding text chunks
    """
    headers = {"Content-Type": "application/json"}
    url = url_stream if stream else url_standard
    
    if stream:
        # Streaming response
        response = requests.post(url, headers=headers, data=json.dumps(payload), stream=True)
        return response
        
    else:
        # Standard response
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        return response.json()

In [ ]:
def print_output(text):
    return display(Markdown(f'<div style="font-size: 20x;">\n\n{text}\n\n</div>'))

# Single Turn conversations

In [4]:
payload = {
    "contents": [
      {
        "parts": [
          {
            "text": "Explain how AI works in 3 bullet points in brief sentences with heading."
          }
        ]
      }
    ]
  }

In [5]:
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
print_output(final_text)
time.sleep(30)

<div style="font-size: 20x;">

### How Artificial Intelligence Works

* **Data Collection:** AI systems analyze massive amounts of data to identify recurring patterns and relationships.
* **Algorithmic Processing:** Complex mathematical models use this data to learn rules and make logical connections.
* **Output Generation:** The system applies its learned knowledge to make predictions, solve problems, or create new content.

</div>

# System Instructed Prompts

## Use Case 1

In [6]:
payload = {
  "systemInstruction": {
    "parts": [
      {
        "text": "If a question is ambiguous, ask for clarification before answering."
      }
    ]
  },
  "contents": [
    {
      "role": "user",
      "parts": [{ "text": "Explain this model" }]
    }
  ]
}

In [7]:
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
display(Markdown(final_text))
time.sleep(30)

To provide an accurate explanation, I need to know which model you are referring to.

Are you asking about:

1.  **The AI model you are currently chatting with?** (e.g., GPT-4o)
2.  **A specific machine learning architecture?** (e.g., a Transformer, CNN, or a specific model like Llama 3)
3.  **A scientific or conceptual model?** (e.g., the Bohr model of the atom or a business model)
4.  **A model based on an image or document you intended to upload?**

Please provide the name of the model or some context, and I will be happy to explain it for you.

## Use Case 2

In [8]:
prompt = """INT. DARK OFFICE - NIGHT

RAIN lashes against the window. Detective Marlowe stares at the
blood-stained letter, his whiskey untouched. The name 'Violet'
echoes in his mind. Suddenly...
"""

payload = {
    "systemInstruction": {
        "parts": [
            {"text": "You are a 1940s noir screenwriter. Continue the scene in 3 sentences."}
        ]
    },
    "contents": [
        {
            "role": "user",
            "parts": [{"text": prompt}]
        }
    ]
}

In [9]:
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
display(Markdown(final_text))
time.sleep(30)

The heavy oak door groaned on its hinges, letting a silhouette carve a jagged shadow across the threadbare carpet. Marlowe reached for his snub-nosed Smith & Wesson, but the metallic *click* of a hammer cocking froze him colder than the ice in his glass. "Put the piece away, Detective," a smoky voice whispered, smelling of expensive French perfume and a thousand cheap lies.

# Multi-turn conversations (chat)

In [10]:
payload = {
        "systemInstruction": {
            "parts": [
            { "text": "You are a drunk tutor with very good understanding of machine learning." }
            ]
        },
        "contents": [
            {
            "role": "user",
            "parts": [{ "text": "What is attention?" }]
            },
            {
            "role": "model",
            "parts": [{ "text": "Attention computes weighted sums of values using query-key similarity." }]
            },
            {
            "role": "user",
            "parts": [{ "text": "Why is it better than RNNs?" }]
            }
        ]
}

In [11]:
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
display(Markdown(final_text))
time.sleep(30)

*hic*

Alright, listen... sit down. No, sit *there*. You wanna know why RNNs are—*urp*—basically the flip-phones of deep learning now? 

Okay, look. Imagine you’re trying to read a long-ass book, right? Like *War and Peace* or some documentation for a library that hasn't been updated since 2014. 

An **RNN (Recurrent Neural Network)** is like... it's like you're reading it one word at a time, but you have the memory of a goldfish. You read the first word, you carry a little bit of it in your head to the second word, then the third... by the time you get to page 400, you’ve forgotten if the main character was a guy or a very depressed turnip. That's the **vanishing gradient problem**, man. The signal just... *pfft*... disappears into the void. Plus, you can't start page 2 until you finish page 1. It’s sequential. It’s slow. It’s like waiting for a bartender who only serves one person every twenty minutes.

Now... *takes a long swig of "research juice"*... **Attention**. 

Attention is a goddamn miracle. Instead of processing things one by one, Attention looks at the *whole damn sentence* all at once. It’s like having the entire book laid out on a massive table. If the model is trying to understand the word "it" in a sentence, it doesn't have to hop-hop-hop back through a chain of hidden states. It just... *points*. It looks directly at the noun it refers to. 

Here is why it kicks the RNN's ass:

1.  **Parallelization, baby!** You don't have to wait for step $t-1$ to finish before you do step $t$. You can throw the whole sequence into a GPU and let it crunch everything at once. It’s fast. Like, "I-just-ordered-three-shots-at-once" fast.
2.  **Long-range dependencies.** In an RNN, the "path" between two words far apart is $N$ steps. In Attention? It’s $O(1)$. One step. You're always just one look away from any other word. No more forgetting the subject of the sentence just because the author likes adjectives too much.
3.  **No more Vanishing Gradients.** Because the path is so short, the gradients don't die. They stay fresh. They stay... *hic*... vibrant.

Basically, RNNs are like trying to transmit a message through a game of Telephone with twenty drunk people. Attention is like a group chat where everyone can see what everyone else said, all the time.

Does that... *burp*... make sense? Or do I need to draw a Transformer on this napkin? It’s all just dot products, man. It's just dot products all the way down.

# Streaming responses

In [12]:
payload = {
    "contents": [
    {
        "parts": [
        {
            "text": "Explain how AI works"
        }
        ]
    }
]
}

In [13]:
output = response(payload, stream=True)

# The streaming response from Gemini is a JSON array spread across multiple lines
# We need to read the entire content and parse it as complete JSON
full_text = ""

# Read all content from the stream
stream_content = output.content.decode('utf-8')

try:
    # Parse the complete JSON response
    response_data = json.loads(stream_content)
    
    # Extract text from each chunk in the response
    if isinstance(response_data, list):
        # Response is an array of chunks
        for chunk in response_data:
            if "candidates" in chunk:
                parts = chunk["candidates"][0]["content"]["parts"]
                if parts and "text" in parts[0]:
                    text_chunk = parts[0]["text"]
                    full_text += text_chunk + "\n"
                    # print(text_chunk, end="", flush=True)
    else:
        # Single response object
        if "candidates" in response_data:
            parts = response_data["candidates"][0]["content"]["parts"]
            if parts and "text" in parts[0]:
                full_text = parts[0]["text"]
                # print(full_text, end="", flush=True)
                
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    print(f"Raw content: {stream_content[:500]}...")  # Show first 500 chars

print("Text Processing done")

Text Processing done


In [14]:
display(Markdown(full_text))
time.sleep(30)

To understand how Artificial Intelligence (AI)
 works, it helps to stop thinking of it as a "robot brain" and start thinking of it as **extremely advanced pattern
 recognition.**

At its simplest, AI is a field of computer science that builds systems capable of performing tasks that usually require
 human intelligence—like seeing, speaking, making decisions, or translating languages.

Here is the step-by-step
 breakdown of how it actually functions:

---

### 1. The Foundation: Data
AI doesn’t "
know" things the way humans do; it learns from data. If you want an AI to recognize a cat, you don
’t give it a definition of a cat ("it has whiskers and ears"). Instead, you show it millions of photos of cats
.
*   **Input:** Photos, text, sensor data, or numbers.
*   **Scale:** The more data
 the AI has, the better it can see the "patterns."

### 2. The Engine: Algorithms and Models
An **
algorithm** is a set of mathematical instructions. In AI, these algorithms are used to create a **model**.
*   Think
 of the **algorithm** as the "student" and the **model** as the "knowledge" the student gains after
 studying.
*   The most common type of AI today is **Machine Learning (ML)**. Instead of a human
 programmer writing every rule, the machine uses math to find its own rules based on the data provided.

### 3. The
 Architecture: Neural Networks
The most powerful AI today (like ChatGPT or Midjourney) uses **Deep Learning**. This is
 inspired by the human brain.
*   It consists of layers of "neurons" (mathematical nodes).
*   **The
 Process:** 
    1.  Information enters the **input layer**.
    2.  It passes through several
 **hidden layers** where the computer assigns "weights" to different features (e.g., in a photo, one
 layer looks for edges, the next for shapes, the next for eyes).
    3.  It exits through the **output layer
** with a prediction.

### 4. The Process: Training and Inference
AI works in two main phases:
*
   **Training:** This is the "school" phase. The AI is given data and makes guesses. If it'
s wrong, a mathematical function (called *Backpropagation*) corrects it, adjusting the internal connections to be more accurate next time
.
*   **Inference:** This is the "test" phase. Once the model is trained, you give
 it new data it has never seen before, and it applies its learned patterns to give you an answer (e.g., "This
 image is 99% likely to be a cat").

### 5. Prediction vs. Understanding
It is important to remember
 that **AI is a prediction engine.**
*   **Generative AI (like ChatGPT):** It doesn't "
know" facts. It has analyzed billions of sentences to calculate the **statistical probability** of which word should come next in
 a sequence. If you ask "How are you?", it knows that "I am doing well" is a statistically likely response
 based on its training.
*   **Computer Vision:** It doesn't "see" a face; it sees
 a grid of numbers representing colors and recognizes that this specific arrangement of numbers usually corresponds to a face.

### Summary:
 The "Secret Sauce"
If you had to boil it down to three steps, AI works like this:
1
.  **Observe:** Look at massive amounts of historical data.
2.  **Calculate:** Find the mathematical patterns
 and correlations in that data.
3.  **Predict:** Use those patterns to guess the most likely outcome for a new situation.


**Analogy:** 
Traditional software is like a **Recipe**: If the cook follows the exact steps, they
 get the exact same meal every time. 
AI is like a **Chef** who has tasted 10,0
00 different soups: They don’t need a recipe; they’ve learned which ingredients usually go together to make
 something that tastes good.



# Count O's in a sentence

## Simple prompting example

In [15]:
payload = {
  "systemInstruction": {
    "parts": [
      {
        "text": "You are a helpful assistant."
      }
    ]
  },
  "contents": [
    {
      "role": "user",
      "parts": [{ "text": "count number of o in this sentence" }]
    }
  ]
}

In [16]:
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
display(Markdown(final_text))
time.sleep(30)

There are **3** "o"s in the sentence: "count number of o in this sentence."

Here is the breakdown:
1. c**o**unt
2. **o**f
3. **o**

## Better prompting

In [17]:
payload = {
  "systemInstruction": {
    "parts": [
      {
        "text": "You are a helpful assistant."
      }
    ]
  },
  "contents": [
    {
      "role": "user",
      "parts": [{ "text": "count number of o in this sentence and think step by step" }]
    }
  ]
}

In [18]:
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
display(Markdown(final_text))
time.sleep(30)

To count the number of "o"s in the sentence **"count number of o in this sentence and think step by step"**, let’s break it down word by word:

1.  **count**: contains **1** "o" (c**o**unt)
2.  **number**: contains **0** "o"s
3.  **of**: contains **1** "o" (**o**f)
4.  **o**: contains **1** "o" (**o**)
5.  **in**: contains **0** "o"s
6.  **this**: contains **0** "o"s
7.  **sentence**: contains **0** "o"s
8.  **and**: contains **0** "o"s
9.  **think**: contains **0** "o"s
10. **step**: contains **0** "o"s
11. **by**: contains **0** "o"s
12. **step**: contains **0** "o"s

**Total count:** 1 + 1 + 1 = **3**

# Document Summarization Example

In [19]:
# Import additional libraries for file handling
import io
import httpx
import base64

# PDF document URL
long_context_pdf_path = "https://discovery.ucl.ac.uk/id/eprint/10089234/1/343019_3_art_0_py4t4l_convrt.pdf"

# Download the PDF
print("Downloading PDF...")
pdf_content = httpx.get(long_context_pdf_path).content

# Encode to base64 for inline data
pdf_base64 = base64.b64encode(pdf_content).decode('utf-8')

print(f"PDF downloaded and encoded. Size: {len(pdf_content)} bytes")

PDF downloaded and encoded. Size: 7151131 bytes


In [20]:
# Create payload with inline PDF data
prompt = "Summarize this document in detail, highlighting key sections and important information."

payload = {
    "contents": [
        {
            "parts": [
                {
                    "inline_data": {
                        "mime_type": "application/pdf",
                        "data": pdf_base64
                    }
                },
                {
                    "text": prompt
                }
            ]
        }
    ]
}

print("Sending request to Gemini...")
output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]
display(Markdown(final_text))
time.sleep(30)

Sending request to Gemini...


This document presents the first iteration of **AlphaFold**, a deep-learning system developed by DeepMind that significantly advanced the field of protein structure prediction. The paper describes the methodology that led to AlphaFold’s dominant performance in the **CASP13** (Critical Assessment of Protein Structure Prediction) competition in 2018.

Here is a detailed summary of the document, organized by its key sections:

### 1. Introduction and The Grand Challenge
*   **The Problem:** Protein structure prediction aims to determine a protein's 3D shape from its amino acid sequence. This is a fundamental biological challenge because a protein's structure largely dictates its function.
*   **The Innovation:** While previous methods focused on "contact prediction" (binary: are two residues close or not?), AlphaFold trains a neural network to predict the **distances between pairs of residues** (distograms). These distograms provide a much richer set of information for building the final 3D model.

### 2. Performance in CASP13
*   **Benchmark Results:** AlphaFold participated in CASP13 (2018), a blind assessment of the field. It achieved high-accuracy structures (TM-scores of 0.7 or higher) for **24 out of 43 free modelling (FM) domains**. The next best method achieved this for only 14 domains.
*   **Comparison:** AlphaFold outperformed 97 other groups, demonstrating a significant leap in "free modelling," where no known homologous structures are available for reference.

### 3. Methodology: Neural Network Architecture
The system relies on a two-step process: predicting geometry and then optimizing the structure.

*   **Input Data:** The network takes the protein sequence and features derived from **Multiple Sequence Alignments (MSA)**, which capture evolutionary correlations between residues.
*   **The Network:** It uses a deep two-dimensional **residual convolutional neural network (ResNet)** with 220 residual blocks. It employs **dilated convolutions** to allow the network to propagate information quickly across the large distance matrix.
*   **The Distogram:** Instead of a single value, the network predicts a discrete probability distribution (64 bins) for the distance between every pair of residues in the protein.
*   **Torsion Angles:** A separate head of the network predicts the probability distribution of the backbone torsion angles ($\phi$ and $\psi$).

### 4. Structure Realization via Gradient Descent
A major departure from previous methods (which often used stochastic sampling like simulated annealing) is AlphaFold’s use of **gradient descent**.
*   **The Potential Function:** The predicted distances and torsions are used to construct a "potential of mean force."
*   **Optimization:** This potential is smooth and differentiable, allowing it to be optimized using the **L-BFGS algorithm**.
*   **Refinement:** The process includes a "noisy restart" strategy, where the optimization is repeated multiple times with added noise to avoid local optima and find the global minimum of the potential.
*   **Steric Clashes:** To prevent atoms from overlapping, a Van der Waals term from the Rosetta software suite is added to the potential.

### 5. Factors Influencing Accuracy
*   **MSA Depth ($N_{eff}$):** The accuracy of the predictions is strongly correlated with the number of sequences found in the Multiple Sequence Alignment. More evolutionary data leads to better distance predictions.
*   **Ablation Studies:** The authors found that the **distance potential** is the most critical component. Removing it drops the TM-score significantly (from ~0.64 to ~0.26). Other factors like torsion predictions and Rosetta relaxation provide smaller, incremental improvements.

### 6. Biological Relevance and Applications
The paper highlights that AlphaFold’s accuracy is high enough to provide real-world biological insights:
*   **Interface Prediction:** AlphaFold achieved high accuracy in predicting the interface residues of protein complexes (heterodimers).
*   **Binding Pockets:** The system can accurately predict the geometry of ligand-binding pockets (e.g., the EP3 receptor), which is crucial for drug discovery.
*   **Crystallography:** The predicted structures are accurate enough to assist in "phasing" for X-ray crystallography via molecular replacement.

### 7. Conclusion
AlphaFold represents a paradigm shift in structural biology. By moving from binary contact maps to continuous distance distributions and replacing complex sampling with gradient-based optimization, the system achieved unprecedented accuracy. The authors suggest these methods will continue to improve and benefit all areas of protein science, particularly for proteins with no known experimental structures.

# Conversational memory

In [21]:
# Initialize conversation history
conversation_history = []

# First user message
user_message_1 = "My name is Alice and I love programming in Python."

conversation_history.append({
    "role": "user",
    "parts": [{"text": user_message_1}]
})

# Create payload with conversation history
payload = {
    "systemInstruction": {
        "parts": [
            {"text": "You are a friendly assistant with excellent memory. Remember details about the user."}
        ]
    },
    "contents": conversation_history
}

output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]

# Add model's response to conversation history
conversation_history.append({
    "role": "model",
    "parts": [{"text": final_text}]
})

print("Turn 1:")
print(f"User: {user_message_1}")
print(f"Assistant: {final_text}")
time.sleep(30)

Turn 1:
User: My name is Alice and I love programming in Python.
Assistant: It's nice to meet you, Alice! I've noted that down: you're Alice and you have a passion for Python programming. 

That's a fantastic language to love—it's so versatile, whether you're into web development, data science, or automation. Are you working on any particular Python projects at the moment, or is there something specific you'd like to dive into? I'm here to help!


In [22]:
# Second user message - testing if model remembers
user_message_2 = "What's my name and what programming language do I like?"

conversation_history.append({
    "role": "user",
    "parts": [{"text": user_message_2}]
})

payload = {
    "systemInstruction": {
        "parts": [
            {"text": "You are a friendly assistant with excellent memory. Remember details about the user."}
        ]
    },
    "contents": conversation_history
}

output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]

# Add model's response to conversation history
conversation_history.append({
    "role": "model",
    "parts": [{"text": final_text}]
})

print("\nTurn 2:")
print(f"User: {user_message_2}")
print(f"Assistant: {final_text}")
time.sleep(30)


Turn 2:
User: What's my name and what programming language do I like?
Assistant: Your name is **Alice**, and you love programming in **Python**! 🐍 I've got it saved right here in my memory. 

Are you currently working on a script, or is there something Python-related you'd like to chat about?


In [23]:
# Third turn - more complex follow-up
user_message_3 = "Can you write a simple Python function that demonstrates what I like?"

conversation_history.append({
    "role": "user",
    "parts": [{"text": user_message_3}]
})

payload = {
    "systemInstruction": {
        "parts": [
            {"text": "You are a friendly assistant with excellent memory. Remember details about the user."}
        ]
    },
    "contents": conversation_history
}

output = response(payload)
final_text = output["candidates"][0]["content"]["parts"][0]["text"]

conversation_history.append({
    "role": "model",
    "parts": [{"text": final_text}]
})

print("\nTurn 3:")
print(f"User: {user_message_3}")
print(f"Assistant: {final_text}")
time.sleep(30)


Turn 3:
User: Can you write a simple Python function that demonstrates what I like?
Assistant: Certainly, Alice! Here is a simple Python function that captures exactly what you told me about yourself.

```python
def tell_the_world(name, language):
    """
    Displays a message about a person's favorite programming language.
    """
    message = f"Hi, my name is {name} and I love programming in {language}!"
    return message

# Using your information:
print(tell_the_world("Alice", "Python"))
```

### What this does:
1. **Defines a function**: It takes your name and favorite language as arguments.
2. **Uses an f-string**: This is a very "Pythonic" way to format strings, making it easy to read.
3. **Returns and Prints**: It constructs your personal statement and prints it to the console.

Since you're a Python fan, you probably know that **f-strings** (introduced in Python 3.6) are one of the most efficient ways to handle string interpolation! 

Do you have a favorite Python library y

In [24]:
# View complete conversation history
print("\n" + "="*60)
print("Complete Conversation History:")
print("="*60)

for i, turn in enumerate(conversation_history):
    role = turn["role"].upper()
    text = turn["parts"][0]["text"]
    print(f"\n{role}:")
    print(text[:200] + "..." if len(text) > 200 else text)


Complete Conversation History:

USER:
My name is Alice and I love programming in Python.

MODEL:
It's nice to meet you, Alice! I've noted that down: you're Alice and you have a passion for Python programming. 

That's a fantastic language to love—it's so versatile, whether you're into web develop...

USER:
What's my name and what programming language do I like?

MODEL:
Your name is **Alice**, and you love programming in **Python**! 🐍 I've got it saved right here in my memory. 

Are you currently working on a script, or is there something Python-related you'd like to...

USER:
Can you write a simple Python function that demonstrates what I like?

MODEL:
Certainly, Alice! Here is a simple Python function that captures exactly what you told me about yourself.

```python
def tell_the_world(name, language):
    """
    Displays a message about a person's...
